In [ ]:
import wandb
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
api = wandb.Api()
entity = "javiermarotomorales"  # set to your entity
projects = ['kdmc_sp0c20', 'kdmc_sm_rml2018', 'kdmc_sbasic', 'kdmc_sawgn2p', 'kdmc_sawgn']

skeys = ['test.acc', 'test.pgd-7_20dB_acc', '_timestamp', 'epoch']
ckeys = ['id', 'lr', 'atk', 'arch', 'loss', 'seed', 'sched', 'dataset', 'n_epochs', 'batch_size', 'dataset_size', 'data_path', 'kt_alpha', 'spr']

ddf = {}
for k in (skeys + ckeys):
    ddf[k] = []
for project in projects:
    runs = api.runs(entity + "/" + project)

    
    for run in tqdm(runs):
        for k in skeys:
            if k in run.summary:
                ddf[k].append(run.summary[k])
            else:
                ddf[k].append(None)
        for k in ckeys:
            if k in run.config:
                ddf[k].append(run.config[k])
            else:
                ddf[k].append(None)

runs_df = pd.DataFrame(ddf)

In [ ]:
entity = "think-team"  # set to your entity
projects = ['kdmc_sp0c20', 'kdmc_sbasic', 'kdmc_sawgn2p']


ddf = {}
for k in (skeys + ckeys):
    ddf[k] = []
for project in projects:
    runs = api.runs(entity + "/" + project)

    
    for run in tqdm(runs):
        for k in skeys:
            if k in run.summary:
                ddf[k].append(run.summary[k])
            else:
                ddf[k].append(None)
        for k in ckeys:
            if k in run.config:
                ddf[k].append(run.config[k])
            else:
                ddf[k].append(None)

runs_df = runs_df.append(pd.DataFrame(ddf))

In [ ]:
runs_df = runs_df.sort_values('_timestamp', ascending=False)

In [ ]:
plt.plot(np.arange(len(runs_df)), runs_df._timestamp)

In [ ]:
#df = runs_df.loc[(runs_df._timestamp > 1.65e9) & (runs_df.epoch == runs_df.n_epochs)]
df = runs_df.loc[(runs_df._timestamp > 1.66243e9) & (runs_df.epoch == runs_df.n_epochs)]
df.loc[df.loss.isin(['std', 'at', 'std_lnr', 'at_lnr']), 'kt_alpha'] = 0
df.loc[df.loss.isin(['std_ml', 'at_ml']), 'kt_alpha'] = 1
df['spr'] = [int(x[2]) for x in df['atk']]
df

In [ ]:
df_plot = df.loc[(df.spr == 20) & (df.dataset == 'sbasic') & ~(df.loss.isin(['at_yml', 'std_yml']))].groupby(['id'], as_index=False).agg({x: ['mean', 'std'] for x in ('test.acc', 'test.pgd-7_20dB_acc')})
df_plot.columns = ["_".join(col_name).rstrip('_') for col_name in df_plot.columns]
df_plot = df_plot.reset_index()
df_plot.loc[df_plot.id == 'std', f"{'test.acc'}_mean"].iloc[0]

In [ ]:
len(df_plot.loc[df_plot.id == 'at']) > 0

In [ ]:
def get_metrics_dataset(dataset, spr=20):
    df_plot = df.loc[(df.spr == 20) & (df.dataset == dataset) & ~(df.loss.isin(['at_yml', 'std_yml']))].groupby(['id'], as_index=False).agg({x: ['mean', 'std'] for x in ('test.acc', 'test.pgd-7_20dB_acc')})
    df_plot.columns = ["_".join(col_name).rstrip('_') for col_name in df_plot.columns]
    df_plot = df_plot.reset_index()
    get_line = lambda r: f"${r['test.acc_mean'].iloc[0]:.2f} \pm {r['test.acc_std'].iloc[0]:.2f}$ & ${r['test.pgd-7_20dB_acc_mean'].iloc[0]:.2f} \pm {r['test.pgd-7_20dB_acc_std'].iloc[0]:.2f}$" if len(r.loc[r.id == t]) > 0 else 'nan'

    with open(f'results_{dataset}.txt', 'w') as f:
        ts = ['std', 'std_lnr', 'std_ml', 'at', 'at_lnr', 'at_ml']
        Ts = ['ST', 'ST\_LNR', 'ST\_ML', 'AT', 'AT\_LNR', 'AT\_ML']
        for t, T in zip(ts, Ts):
            f.write(f"		{T} & {get_line(df_plot.loc[df_plot.id == t])} \\\\ \n")

get_metrics_dataset('sbasic')
get_metrics_dataset('sawgn2p')
get_metrics_dataset('sp0c20')

In [ ]:
def get_metrics_dataset(dataset, spr=20):
    df_plot = df.loc[(df.spr == 20) & (df.dataset == dataset) & ~(df.loss.isin(['at_yml', 'std_yml']))].groupby(['id'], as_index=False).agg({x: ['mean', 'std'] for x in ('test.acc', 'test.pgd-7_20dB_acc')})
    df_plot.columns = ["_".join(col_name).rstrip('_') for col_name in df_plot.columns]
    df_plot = df_plot.reset_index()
    get_line_t = lambda r, m, t: f"${r.loc[r.id == t, f'{m}_mean'].iloc[0]:.2f} \pm {r.loc[r.id == t, f'{m}_std'].iloc[0]:.2f}$" if len(r.loc[r.id == t]) > 0 else 'nan'
    get_line = lambda r, m, ts: f"{m} & {' & '.join([get_line_t(r, m, t) for t in ts])} \\\\ \n"

    with open(f'results_{dataset}.txt', 'w') as f:
        ts = ['std', 'std_lnr', 'std_ml', 'at', 'at_lnr', 'at_ml']
        f.write(get_line(df_plot, 'test.acc', ts))
        f.write(get_line(df_plot, 'test.pgd-7_20dB_acc', ts))

get_metrics_dataset('sbasic')
get_metrics_dataset('sawgn2p')
get_metrics_dataset('sp0c20')

In [ ]:
def get_metrics(spr=20):
    df_plot = df.loc[(df.spr == 20) & ~(df.loss.isin(['at_yml', 'std_yml']))].groupby(['id', 'dataset'], as_index=False).agg({x: ['mean', 'std'] for x in ('test.acc', 'test.pgd-7_20dB_acc')})
    df_plot.columns = ["_".join(col_name).rstrip('_') for col_name in df_plot.columns]
    df_plot = pd.pivot(df_plot, index=('id'), columns=['dataset'], values=['test.acc_mean', 'test.pgd-7_20dB_acc_mean', 'test.acc_std', 'test.pgd-7_20dB_acc_std']).reset_index()
    get_line_d = lambda r, d: f"${r['test.acc_mean'][d]:.2f} \pm {r['test.acc_std'][d]:.2f}$ & ${r['test.pgd-7_20dB_acc_mean'][d]:.2f} \pm {r['test.pgd-7_20dB_acc_std'][d]:.2f}$"
    get_line = lambda r: f"{r['id'].values[0]} & {get_line_d(r, 'sbasic')} & {get_line_d(r, 'sawgn2p')} & {get_line_d(r, 'sp0c20')} \\\\ \n"

    with open(f'results.txt', 'w') as f:
        for i in trange(len(df_plot)):
            r = df_plot.iloc[i]
            f.write(get_line(r))

get_metrics()
    

In [ ]:
df_plot = df.loc[(df.spr == 20) & (df.dataset == 'sp0c20') & ~(df.loss.isin(['at_yml', 'std_yml']))]
df_plot[['id', 'seed', 'test.acc', 'test.pgd-7_20dB_acc']].sort_values(['id', 'seed'])

In [ ]:
def plot_std_metrics(dataset, lims, rang=10):
    df_plot = df.loc[(df.dataset == dataset) & (df.id.isin(['std', 'std_lnr', 'std_ml']))].groupby(['id', 'kt_alpha'], as_index=False).mean()
    fig1, axs = plt.subplots(ncols=2, figsize=(12, 4))
    sns.barplot(data=df_plot, x='test.acc', y='id', orient='h', ax=axs[0])
    axs[0].set_xlim((lims[0], lims[0] + rang))
    axs[0].set(ylabel=None)
    axs[1] = sns.barplot(data=df_plot, x='test.pgd-7_20dB_acc', y='id', orient='h')
    axs[1].set_xlim((lims[1], lims[1] + rang))
    axs[1].set(ylabel=None)

def plot_std_alpha(dataset):
    df_plot = df.loc[(df.dataset == dataset) & (df.id.isin(['std', 'std_ml', 'std_yml']))].groupby(['id', 'kt_alpha'], as_index=False).mean().sort_values('kt_alpha')
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(df_plot['kt_alpha'], df_plot['test.acc'], '-', color='blue')
    ax2.plot(df_plot['kt_alpha'], df_plot['test.pgd-7_20dB_acc'], '-', color='orange')

    ax1.set_xlabel('alpha')
    ax1.set_ylabel('Accuracy', color='b')
    ax2.set_ylabel('Robustness', color='orange')

def plot_at_metrics(dataset, spr, lims, rang=10):
    df_plot = df.loc[(df.dataset == dataset) & (df.id.isin(['at', 'at_lnr', 'at_ml'])) & (df.spr == spr)].groupby(['id', 'kt_alpha', 'spr'], as_index=False).mean()
    if len(df_plot) == 0:
        return
    fig1, axs = plt.subplots(ncols=2, figsize=(12, 4))
    sns.barplot(data=df_plot, x='test.acc', y='id', orient='h', ax=axs[0])
    axs[0].set_xlim((lims[0], lims[0] + rang))
    axs[0].set(ylabel=None)
    axs[1] = sns.barplot(data=df_plot, x='test.pgd-7_20dB_acc', y='id', orient='h')
    axs[1].set_xlim((lims[1], lims[1] + rang))
    axs[1].set(ylabel=None)

def plot_at_alpha(dataset, spr):
    df_plot = df.loc[(df.dataset == dataset) & (df.id.isin(['at', 'at_ml', 'at_yml'])) & (df.spr == spr)].groupby(['id', 'kt_alpha'], as_index=False).mean().sort_values('kt_alpha')
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(df_plot['kt_alpha'], df_plot['test.acc'], '-', color='blue')
    ax2.plot(df_plot['kt_alpha'], df_plot['test.pgd-7_20dB_acc'], '-', color='orange')

    ax1.set_xlabel('alpha')
    ax1.set_ylabel('Accuracy', color='b')
    ax2.set_ylabel('Robustness', color='orange')

In [ ]:
plot_std_metrics('sbasic', lims=(80, 20))
plot_std_alpha('sbasic')
plot_at_metrics('sbasic', spr=30, lims=(80, 38))
plot_at_metrics('sbasic', spr=25, lims=(70, 48))
plot_at_metrics('sbasic', spr=20, lims=(58, 52))
plot_at_alpha('sbasic', spr=30)
plot_at_alpha('sbasic', spr=25)
plot_at_alpha('sbasic', spr=20)

In [ ]:
plot_std_metrics('sawgn2p', lims=(80, 20))
plot_std_alpha('sawgn2p')
plot_at_metrics('sawgn2p', spr=30, lims=(80, 38))
plot_at_metrics('sawgn2p', spr=25, lims=(70, 48))
plot_at_metrics('sawgn2p', spr=20, lims=(58, 52))
plot_at_alpha('sawgn2p', spr=30)
plot_at_alpha('sawgn2p', spr=25)
plot_at_alpha('sawgn2p', spr=20)

In [ ]:
plot_std_metrics('sp0c20', lims=(72, 14))
plot_std_alpha('sp0c20')

In [ ]:
df.loc[(df.dataset == 'sp0c20') & (df.id.str.startswith('at'))]

In [ ]:
def plot_at_metrics(dataset, spr, lims, rang=10):
    df_plot = df.loc[(df.dataset == dataset) & (df.id.isin(['at', 'at_lnr', 'at_ml', 'at_aml'])) & (df.spr == spr)].groupby(['id', 'kt_alpha', 'spr'], as_index=False).mean()
    fig1, axs = plt.subplots(ncols=2, figsize=(12, 4))
    sns.barplot(data=df_plot, x='test.acc', y='id', orient='h', ax=axs[0])
    axs[0].set_xlim((lims[0], lims[0] + rang))
    axs[0].set(ylabel=None)
    axs[1] = sns.barplot(data=df_plot, x='test.pgd-7_20dB_acc', y='id', orient='h')
    axs[1].set_xlim((lims[1], lims[1] + rang))
    axs[1].set(ylabel=None)

plot_at_metrics('sp0c20', spr=30, lims=(70, 34))
plot_at_metrics('sp0c20', spr=25, lims=(60, 44))

In [ ]:
plot_at_alpha('sp0c20', spr=30)
plot_at_alpha('sp0c20', spr=25)

In [ ]:
# Latex code
get_line = lambda r: f"{r['dataset'][0]} & {r['arch'][0]}-d{int(1 / r['lwidth'][0])} & {r['sched'][0]}-{int(r['n_epochs'][0])} & ${r['test.acc']['mean']:.2f} \pm {r['test.acc']['std']:.2f}$ & ${r['test.autoattack_acc']['mean']:.2f} \pm {r['test.autoattack_acc']['std']:.2f}$ \\\\ \n"

with open('results.txt', 'w') as f:
    for i in trange(len(res)):
        r = res.iloc[i]
        f.write(get_line(r))